<a href="https://colab.research.google.com/github/pkyle89/DS-Unit-2-Linear-Models/blob/master/DS10_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Ridge Regression**
Assignment

*We're going back to our other New York City real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.*

But not just for condos in Tribeca...

[ ] Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

[ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

[ ] Do one-hot encoding of categorical features.

[ ] Do feature selection with SelectKBest.

[ ] Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

[ ] Get mean absolute error for the test set.

[ ] As always, commit your notebook to your fork of the GitHub repo.
The NYC Department of Finance has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the NYC OpenData portal.


**Stretch Goals**

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

[ ] Add your own stretch goal(s) !

[ ] Instead of Ridge, try LinearRegression. Depending on how many features you select, your errors will probably blow up! 💥

[ ] Instead of Ridge, try RidgeCV.

[ ] Learn more about feature selection:
"Permutation importance"
scikit-learn's User Guide for Feature Selection
mlxtend library
scikit-learn-contrib libraries: boruta_py & stability-selection
Feature Engineering and Selection by Kuhn & Johnson.

[ ] Try statsmodels if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.

[ ] Read An Introduction to Statistical Learning, Chapters 1-3, for more math & theory, but in an accessible, readable way.

[ ] Try scikit-learn pipelines.



In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import numpy as np
from math import factorial
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [98]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
#Use a subset of the data where 
#BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' 
#and the sale price was more than 100 thousand and less than 2 million.

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df[(100000) < df['SALE_PRICE'] & (df['SALE_PRICE'] < 2000000)]

In [100]:
df = df.fillna(value=df.mean(skipna=True))
df

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE


In [101]:
#Do train/test split. Use data from January — March 2019 to train.
#Use data from April 2019 to test.

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].describe()

count       0
unique      0
top       NaN
freq      NaN
Name: SALE_DATE, dtype: object

In [102]:
df['SALE_DATE'].dt.month.value_counts()

Series([], Name: SALE_DATE, dtype: int64)

In [103]:

train = df[(df.SALE_DATE.dt.month >= 1) & (df.SALE_DATE.dt.month <= 3)]
test  = df[df.SALE_DATE.dt.month == 4]
train.shape, test.shape


((0, 21), (0, 21))

In [104]:
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'BUILDING_CLASS_AT_PRESENT',
       'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [0]:
#Do one-hot encoding of categorical features.
target = 'SALE_PRICE'
high_cardinality = ['LOT', 'SALE_DATE', 'EASE-MENT', 'ADDRESS', 'APARTMENT_NUMBER']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
X_test = test[target]

y_train = train[features]
y_test = train[target]


import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)


In [106]:
X_train

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE


In [0]:
#Do feature selection with SelectKBest.

def engineer_features(X):
    
    # Avoid SettingWithCopyWarning
    X = X.copy()
    
    # How many total perks does each apartment have?
    perk_cols = ['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'GROSS_SQUARE_FEET',
                 'YEAR_BUILT', 'BOROUGH']
    X['perk_count'] = X[perk_cols].sum(axis=1)

        
    # Total number of rooms (beds + baths)
    X['total_pro'] = X['RESIDENTIAL_UNITS'] + X['COMMERCIAL_UNITS']

    return X

#X_train = engineer_features(X_train)
#X_test = engineer_features(X_test)

In [108]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=15)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

ValueError: ignored

In [110]:
#Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)
#https://scikit-learn.org/stable/modules/preprocessing.html
#Get mean absolute error for the test set.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')


1 features


ValueError: ignored

In [0]:
#As always, commit your notebook to your fork of the GitHub repo.


